# Plot Integration Agent 테스트 (Production Level)

플롯 통합 에이전트 (Level 2) 테스트 노트북

## 역할: "Story Structure Architect" (스토리 구조 설계자)
- 이벤트 기반 3-Act 구조 분석
- 복선(Foreshadowing) 감지 및 연결
- **🆕 Tension Curve 배열** - 오디오/연출 타이밍용
- **🆕 Narrative Beats** - 컷 편집/삽화 프롬프트용
- Neo4j-ready 출력 (FORESHADOWS 엣지)

> **Note**: Event Agent의 출력을 받아 스토리 구조를 분석합니다.

---

### 💡 Temporal Control 출력 구조

| 필드 | 설명 | 용도 |
|------|------|------|
| `tension_curve` | [3, 5, 7, 8, 6] 배열 | 오디오 빌드업/드롭 자동 생성 |
| `narrative_beats` | 비트 단위 분할 | 컷 연출/삽화 프롬프트 |
| `beat_type` | SETUP, CLIMAX 등 | 연출 유형 결정 |
| `visual_prompt` | 삽화용 프롬프트 | 이미지 AI 직접 입력 |

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
# 시뮬레이션: Event Agent에서 추출된 이벤트 목록
EXTRACTED_EVENTS = [
    {"event_id": "E001", "description": "서진과 하나가 어두운 숲에서 만남", 
     "event_type": "action", "importance": 7, "participants": ["서진", "하나"]},
    {"event_id": "E002", "description": "이민호가 갑자기 나타남", 
     "event_type": "revelation", "importance": 9, "participants": ["이민호"]},
    {"event_id": "E003", "description": "서진과 이민호의 대화 - 과거 우정 언급", 
     "event_type": "dialogue", "importance": 8, "participants": ["서진", "이민호"]},
    {"event_id": "E004", "description": "이민호가 과거 배신의 이유를 암시", 
     "event_type": "revelation", "importance": 8, "participants": ["이민호"]},
    {"event_id": "E005", "description": "하나가 경비병 접근을 경고", 
     "event_type": "action", "importance": 6, "participants": ["하나"]}
]

EXTRACTED_CHARACTERS = [
    {"name": "서진", "role": "protagonist"},
    {"name": "하나", "role": "supporting"},
    {"name": "이민호", "role": "antagonist"}
]

RELATIONSHIP_GRAPH = {
    "relationships": [
        {"source": "이민호", "target": "서진", "relation_type": "BETRAYED", "bidirectional": False},
        {"source": "서진", "target": "하나", "relation_type": "FRIENDLY", "bidirectional": True}
    ]
}

def create_base_state():
    return {
        "content": "테스트 스토리", "project_id": "test-001", "document_id": "doc-001", 
        "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
        "extracted_events": EXTRACTED_EVENTS,
        "extracted_characters": EXTRACTED_CHARACTERS,
        "relationship_graph": RELATIONSHIP_GRAPH,
        "plot_integration": {}, "messages": [], "errors": []
    }

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

## 1. 플롯 통합 분석 테스트

In [3]:
from app.agents.analysis.plot import plot_integration_node

async def test_plot_integration():
    print("📖 Plot Integration 테스트...")
    state = create_base_state()
    return await plot_integration_node(state)

result = run_async(test_plot_integration())
report = result.get("plot_integration", {})

print(f"\n" + "="*70)
print("📖 Plot Integration 결과")
print("="*70)

📖 Plot Integration 테스트...
[PLOT] Analyzing 5 events for narrative structure
[PLOT] Available event_ids: ['E001', 'E002', 'E003', 'E004', 'E005']
[PLOT] Beats: 5, Tension curve: [6, 8, 9, 9, 7]
[PLOT] Foreshadowing: 1, Acts: 3

📖 Plot Integration 결과


## 2. 🆕 Tension Curve (긴장도 곡선 배열)

In [4]:
print("="*70)
print("📈 Tension Curve (오디오/연출 타이밍용)")
print("="*70)

tension_curve = report.get("tension_curve", [])
print(f"\n배열 데이터: {tension_curve}")
print(f"길이: {len(tension_curve)} 비트")

if tension_curve:
    print(f"\n시각화:")
    max_tension = max(tension_curve) if tension_curve else 10
    for i, t in enumerate(tension_curve):
        bar = '█' * t + '░' * (10 - t)
        peak = ' ★ PEAK' if t == max_tension else ''
        print(f"Beat {i+1}: [{bar}] {t}/10{peak}")
    
    # 변화 패턴 분석
    changes = []
    for i in range(1, len(tension_curve)):
        diff = tension_curve[i] - tension_curve[i-1]
        if diff > 0:
            changes.append("↑")
        elif diff < 0:
            changes.append("↓")
        else:
            changes.append("→")
    print(f"\n변화 패턴: {' '.join(changes)}")
    print(f"\n💡 오디오 엔진 활용: 빌드업({'↑' in changes}), 드롭({'↓' in changes})")

📈 Tension Curve (오디오/연출 타이밍용)

배열 데이터: [6, 8, 9, 9, 7]
길이: 5 비트

시각화:
Beat 1: [██████░░░░] 6/10
Beat 2: [████████░░] 8/10
Beat 3: [█████████░] 9/10 ★ PEAK
Beat 4: [█████████░] 9/10 ★ PEAK
Beat 5: [███████░░░] 7/10

변화 패턴: ↑ ↑ → ↓

💡 오디오 엔진 활용: 빌드업(True), 드롭(True)


## 3. 🆕 Narrative Beats (비트 단위 분할)

In [5]:
print("="*70)
print("🎬 Narrative Beats (컷 편집/삽화용)")
print("="*70)

beats = report.get("narrative_beats", [])
print(f"\n총 {len(beats)}개 비트 감지")

for beat in beats:
    bid = beat.get('beat_id', '?')
    text = beat.get('text', '')
    beat_type = beat.get('beat_type', 'UNKNOWN')
    event_ref = beat.get('event_ref', 'N/A')
    visual = beat.get('visual_prompt', '')
    
    # Beat type에 따른 아이콘
    icons = {
        'SETUP': '🏠', 'INCITING_INCIDENT': '⚡', 'RISING_ACTION': '📈',
        'CLIMAX': '💥', 'FALLING_ACTION': '📉', 'RESOLUTION': '🏁'
    }
    icon = icons.get(beat_type, '🔹')
    
    print(f"\n{icon} Beat {bid}: [{beat_type}]")
    print(f"   텍스트: {text}")
    print(f"   이벤트: {event_ref}")
    print(f"   🎨 Visual Prompt: {visual}")

🎬 Narrative Beats (컷 편집/삽화용)

총 5개 비트 감지

🏠 Beat 1: [SETUP]
   텍스트: 서진과 하나가 어두운 숲에서 만남
   이벤트: E001
   🎨 Visual Prompt: Two figures meeting in dark forest, mysterious atmosphere

⚡ Beat 2: [INCITING_INCIDENT]
   텍스트: 이민호의 갑작스러운 등장
   이벤트: E002
   🎨 Visual Prompt: Shadowy figure emerging from darkness, tense moment

📈 Beat 3: [RISING_ACTION]
   텍스트: 서진과 이민호의 대화 - 과거 우정 언급
   이벤트: E003
   🎨 Visual Prompt: Two men in intense conversation, flashbacks of past

📈 Beat 4: [RISING_ACTION]
   텍스트: 이민호가 과거 배신의 이유를 암시
   이벤트: E004
   🎨 Visual Prompt: Shadowy figure revealing clues about past betrayal

📉 Beat 5: [FALLING_ACTION]
   텍스트: 하나가 경비병 접근을 경고
   이벤트: E005
   🎨 Visual Prompt: Woman urgently warning others of approaching danger


## 4. Multimedia Pipeline Summary

In [6]:
print("="*70)
print("🎬 Multimedia Pipeline Summary")
print("="*70)

mm_summary = report.get("multimedia_summary", {})
print(json.dumps(mm_summary, ensure_ascii=False, indent=2))

print(f"\n✅ 검증:")
print(f"   - Beat 수: {mm_summary.get('beat_count', 0)}")
print(f"   - Tension Curve 길이: {mm_summary.get('tension_curve_length', 0)}")
print(f"   - 일치 여부: {'✅ 일치' if mm_summary.get('beat_count') == mm_summary.get('tension_curve_length') else '⚠️ 불일치'}")
print(f"   - Visual Prompts 완성: {'✅' if mm_summary.get('has_visual_prompts') else '⚠️ 누락'}")

🎬 Multimedia Pipeline Summary
{
  "beat_count": 5,
  "tension_curve_length": 5,
  "tension_curve_source": "llm",
  "tension_range": {
    "min": 6,
    "max": 9,
    "peak_index": 2
  },
  "beat_types": [
    "FALLING_ACTION",
    "INCITING_INCIDENT",
    "RISING_ACTION",
    "SETUP"
  ],
  "has_visual_prompts": true
}

✅ 검증:
   - Beat 수: 5
   - Tension Curve 길이: 5
   - 일치 여부: ✅ 일치
   - Visual Prompts 완성: ✅


## 5. Foreshadowing (복선)

In [7]:
print("="*70)
print("🔮 Foreshadowing (복선)")
print("="*70)

for fs in report.get("foreshadowing", []):
    fid = fs.get('foreshadow_id', 'Unknown')
    source = fs.get('source_event', 'N/A')
    target = fs.get('target_event', '(미래 이벤트)')
    hint = fs.get('hint_text', '')
    confidence = fs.get('confidence', 0)
    
    print(f"\n[{fid}] {source} → {target}")
    print(f"   힌트: {hint}")
    print(f"   신뢰도: {'⭐' * min(confidence, 10)} ({confidence}/10)")

🔮 Foreshadowing (복선)

[F001] E004 → None
   힌트: 이민호가 과거 배신의 이유를 암시
   신뢰도: ⭐⭐⭐⭐⭐⭐⭐⭐ (8/10)


## 6. 삽화 생성 시뮬레이션

In [8]:
print("="*70)
print("🎨 삽화 생성 AI 입력 시뮬레이션")
print("="*70)

beats = report.get("narrative_beats", [])
tension_curve = report.get("tension_curve", [])

print("\n각 비트별 이미지 AI 프롬프트:")
for i, beat in enumerate(beats):
    tension = tension_curve[i] if i < len(tension_curve) else 5
    mood = "intense, dramatic" if tension >= 7 else ("calm, peaceful" if tension <= 3 else "neutral")
    
    visual = beat.get('visual_prompt', '')
    enhanced = f"{visual}, {mood}, tension level {tension}/10"
    
    print(f"\n[Beat {beat.get('beat_id')}]")
    print(f"   기본: {visual}")
    print(f"   강화: {enhanced}")

🎨 삽화 생성 AI 입력 시뮬레이션

각 비트별 이미지 AI 프롬프트:

[Beat 1]
   기본: Two figures meeting in dark forest, mysterious atmosphere
   강화: Two figures meeting in dark forest, mysterious atmosphere, neutral, tension level 6/10

[Beat 2]
   기본: Shadowy figure emerging from darkness, tense moment
   강화: Shadowy figure emerging from darkness, tense moment, intense, dramatic, tension level 8/10

[Beat 3]
   기본: Two men in intense conversation, flashbacks of past
   강화: Two men in intense conversation, flashbacks of past, intense, dramatic, tension level 9/10

[Beat 4]
   기본: Shadowy figure revealing clues about past betrayal
   강화: Shadowy figure revealing clues about past betrayal, intense, dramatic, tension level 9/10

[Beat 5]
   기본: Woman urgently warning others of approaching danger
   강화: Woman urgently warning others of approaching danger, intense, dramatic, tension level 7/10


## 7. Spring Boot Callback 데이터 구조

In [9]:
print("="*70)
print("📤 Spring Boot Callback 데이터 구조")
print("="*70)

callback_data = {
    "plot_integration": {
        "plot_summary": report.get("plot_summary", {}),
        "overall_tension": report.get("overall_tension", 5),
        "tension_curve": report.get("tension_curve", []),  # 🆕 Array
        "narrative_beats": report.get("narrative_beats", []),  # 🆕 Beats
        "multimedia_summary": report.get("multimedia_summary", {}),
        "foreshadowing_count": len(report.get("foreshadowing", [])),
        "neo4j_foreshadowing": report.get("neo4j_foreshadowing", [])
    }
}

print(json.dumps(callback_data, ensure_ascii=False, indent=2))

📤 Spring Boot Callback 데이터 구조
{
  "plot_integration": {
    "plot_summary": {
      "narrative": "서진과 하나가 어두운 숲에서 만나던 중, 갑자기 이민호가 나타난다. 이민호와 서진은 과거 우정에 대해 대화를 나누고, 이민호는 과거 배신의 이유를 암시한다. 한편 하나는 경비병의 접근을 경고한다.",
      "central_conflict": "서진과 이민호 사이의 과거 배신과 갈등"
    },
    "overall_tension": 8,
    "tension_curve": [
      6,
      8,
      9,
      9,
      7
    ],
    "narrative_beats": [
      {
        "beat_id": 1,
        "text": "서진과 하나가 어두운 숲에서 만남",
        "beat_type": "SETUP",
        "event_ref": "E001",
        "visual_prompt": "Two figures meeting in dark forest, mysterious atmosphere"
      },
      {
        "beat_id": 2,
        "text": "이민호의 갑작스러운 등장",
        "beat_type": "INCITING_INCIDENT",
        "event_ref": "E002",
        "visual_prompt": "Shadowy figure emerging from darkness, tense moment"
      },
      {
        "beat_id": 3,
        "text": "서진과 이민호의 대화 - 과거 우정 언급",
        "beat_type": "RISING_ACTION",
        "event_ref": "E003",
        "visual_prompt": "